# IMPORTACION  DE LIBRERIAS

In [12]:
import collections
import random
import simpy
from tkinter import *
from tkinter import ttk
from tkinter import messagebox

# DEFINICION DE LOS METODOS 

In [13]:
def ventaBoletos(env, num_boletos, pelicula, teatro):
  with teatro.contador.request() as turno:
    resultado = yield turno | teatro.sold_out[pelicula]
    if turno not in resultado:
      teatro.num_renegados[pelicula] += 1
      return
    if teatro.num_boletos[pelicula] < num_boletos:
      yield env.timeout(0.5) 
      return
    teatro.num_boletos[pelicula] -= num_boletos
    if teatro.num_boletos[pelicula] < 2:
      teatro.sold_out[pelicula].succeed()
      teatro.tiempo_agotado[pelicula] = env.now
      teatro.num_boletos[pelicula] = 0
    yield env.timeout(1)


def llegadaClientes(env, teatro):
  while True:
    yield env.timeout(random.expovariate(1/0.5))
    pelicula = random.choices(teatro.peliculas, teatro.probabilidad, k=1)
    num_boletos = random.randint(1, 6)
    if teatro.num_boletos[pelicula[0]]:
      env.process(ventaBoletos(env, num_boletos, pelicula[0], teatro))


def process_run(vel,nboleto,tsimulacion):
    
    VELOCIDAD_LLEGADA = vel
    NUM_BOLETO = nboleto
    TIEMPO_SIMULACION = tsimulacion
    
    resultado = ""
    
    Teatro = collections.namedtuple('Teatro', 'contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados')

    print('MALL DEL RIO')
    
    resultado = "MALL DEL RIO"
    env = simpy.Environment()

    contador = simpy.Resource(env,capacity=1)
    peliculas = ['El amanecer de los muertos', 'Spiderman', 'Chuky']
    
    resultado = resultado +"\n"
    
    probabilidad=[0.1, 0.3, 0.6]
    num_boletos = {pelicula: NUM_BOLETO for pelicula in peliculas}
    sold_out = {pelicula: env.event() for pelicula in peliculas}
    tiempo_agotado = {pelicula: None for pelicula in peliculas}
    num_renegados = {pelicula: 0 for pelicula in peliculas}

    teatro = Teatro(contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados)
    env.process(llegadaClientes(env, teatro))
    env.run(until=TIEMPO_SIMULACION)

    # Analisis y resultados
    for pelicula in peliculas:
        if teatro.sold_out[pelicula]:
            print('Pelicula: %s se agoto en  %.1f' %(pelicula, teatro.tiempo_agotado[pelicula]))
            sms1 = 'Pelicula: %s se agoto %.1f' %(pelicula, teatro.tiempo_agotado[pelicula])
            
            print(' personas  renegadas %s' %teatro.num_renegados[pelicula])
            sms2 = 'personas renegadas %s' %teatro.num_renegados[pelicula]
            
            resultado = resultado + "\n" + sms1 +"\n"+ sms2
    return resultado

# EJECUCION DE PROCESO SIN VENTANA

In [14]:
resp = process_run(vel=80,nboleto=50,tsimulacion=300)
print(resp)

MALL DEL RIO
Pelicula: El amanecer de los muertos se agoto en  61.5
 personas  renegadas 0
Pelicula: Spiderman se agoto en  38.2
 personas  renegadas 4
Pelicula: Chuky se agoto en  21.2
 personas  renegadas 14
MALL DEL RIO

Pelicula: El amanecer de los muertos se agoto 61.5
personas renegadas 0
Pelicula: Spiderman se agoto 38.2
personas renegadas 4
Pelicula: Chuky se agoto 21.2
personas renegadas 14


In [15]:
ventana = Tk()
ventana.geometry("300x200")
ventana.title("MALL DEL RIO")
ventana.resizable(0,0)

txvelocidad=StringVar()
velocidad = Label(ventana,text="Velocidad", width=10).place(x=15, y=10)
txtvelocidad = ttk.Entry(ventana, textvariable=txvelocidad).place(x=150,y=10)

txnboleto=StringVar()
nboleto = Label(ventana,text="# boletos", width=20).place(x=5, y=40)
txtnboleto = ttk.Entry(ventana, textvariable=txnboleto).place(x=150,y=40)

txsimulacion=StringVar()
simulacion = Label(ventana,text="Tiempo", width=20).place(x=10, y=70)
txtsimulacion = ttk.Entry(ventana,textvariable=txsimulacion).place(x=150,y=70)


def guardar():
    vel = txvelocidad.get()
    nbol=txnboleto.get()
    sim=txsimulacion.get()
    
    res = process_run(vel=int(vel),nboleto=int(nbol),tsimulacion=int(sim))
    print("resp ",res)
    #tk.messagebox.showinfo(title="Resultado de simulacion", message=res, **options)
    messagebox.showinfo(message=res,title="Resultado de simulación")
    

Button(ventana, text="Correr", command=guardar).place(x=100, y=150)

ventana.mainloop()